# Implementing a Convolutional Neural Network on the CPU from Scratch
I could implement a CNN using pytorch's layers with trivial ease `nn.Conv2d` is all that is required. However, I'd like to demonstrate a detailed understanding of the fundamental mechanics.

In [14]:
import torch 
from torch import optim, nn

from PIL import Image
import numpy as np
import cv2

import pandas as pd
import os

#### CSV to png

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


It's clear each row represents an image. I want to convert the underlying data structure, `numpy.ndarray` to a PNG file so that I may augment and manipulate the images with imaging libraries, like `PIL`.

In [21]:
for i in range(len(train_df)):
    arr = np.array(train_df.iloc[i,1:]).reshape(28,28)
    arr = arr.astype('uint8')
    #arr = arr / 255    # This is for floats
    #display(arr)
    img = Image.fromarray(arr, mode='L')
    #img.show()
    label = train_df.iloc[i,0]
    f_path = f'./train/{label}/train_{i}.png'
    os.makedirs(f_path, exist_ok=True)
    img.save(f_path)